# **Does hosting the Olympics improve performance on the medals table?**



In [1]:
from google.colab import files

uploaded = files.upload()


Saving olympics.csv to olympics.csv


## Countries who hosted atleast 2 times

In [9]:
import pandas as pd
olympics = pd.read_csv('olympics.csv')
host_data = []

for city in olympics['City'].unique():
    host_count = 0
    for year in olympics['Year'].unique():
        if len(olympics[(olympics['City'] == city) & (olympics['Year'] == year)]) > 0:
            host_count += 1
    host_data.append([city, host_count])

host_data = pd.DataFrame(host_data,columns = ['City','No. of times hosted']).sort_values(by='No. of times hosted', ascending = False)

host_data.reset_index(inplace = True, drop = True)

host_data.head(10)

,City,No. of times hosted
0,London,3
1,Athina,3
2,Innsbruck,2
3,Sankt Moritz,2
4,Paris,2
5,Los Angeles,2
6,Lake Placid,2
7,Stockholm,2
8,Amsterdam,1
9,Berlin,1


from matplotlib import pyplot as plt
_df_9['No. of times hosted'].plot(kind='hist', bins=20, title='No. of times hosted')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_10['No. of times hosted'].plot(kind='line', figsize=(8, 4), title='No. of times hosted')
plt.gca().spines[['top', 'right']].set_visible(False)

So ["UK", "Greece", "Austria", "Switzerland", "France", "USA", "Sweden"], these are the countries associated with the cities which have hosted the olympics 2 or more times. Let's map these countries with the cities.

In [10]:
countries_cities_map = {"USA": ["Lake Placid", "Los Angeles"], "UK": ["London"], "Greece": ["Athina"], "Austria": ["Innsbruck"], "Switzerland": ["Sankt Moritz"], "France": ["Paris"], "Sweden":["Stockholm"]}

countries = ["USA", "UK", "Greece", "Austria", "Switzerland", "France", "Sweden"]
no_of_times = [4, 3, 3, 2, 2, 2, 2]

countries_hosted = pd.DataFrame(data=zip(countries,no_of_times), columns = ['Country','Hosted']).sort_values(by='Hosted', ascending = False)

countries_hosted

,Country,Hosted
0,USA,4
1,UK,3
2,Greece,3
3,Austria,2
4,Switzerland,2
5,France,2
6,Sweden,2


## Total medal tally of these countries

In [11]:
olympics.rename(columns={'region':'Country'}, inplace=True)

selected_countries_medals = olympics.loc[(olympics["Country"].isin(countries)) & (olympics["Medal"] != "DNW")]

selected_countries_medals.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Country,notes
10,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 10 kilometres,NaN,USA,NaN
11,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 50 kilometres,NaN,USA,NaN
12,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 10/15 kilometres Pu...,NaN,USA,NaN
13,6,Per Knut Aaland,M,31.0,188.0,75.0,United States,USA,1992 Winter,1992,Winter,Albertville,Cross Country Skiing,Cross Country Skiing Men's 4 x 10 kilometres R...,NaN,USA,NaN
14,6,Per Knut Aaland,M,33.0,188.0,75.0,United States,USA,1994 Winter,1994,Winter,Lillehammer,Cross Country Skiing,Cross Country Skiing Men's 10 kilometres,NaN,USA,NaN


## **Let's analyse the medal tally of USA**

In [14]:
usa_hosted_medals = selected_countries_medals[selected_countries_medals["City"].isin(["Los Angeles", "Lake Placid"])]
usa_hosted_medals = usa_hosted_medals[usa_hosted_medals["Country"] == "USA"]

usa_hosted = usa_hosted_medals.groupby("Year")["Medal"].count()

usa_hosted

Year
1932    223
1980     30
1984    352
Name: Medal, dtype: int64

In [16]:
usa_not_hosted_medals = selected_countries_medals[~selected_countries_medals["City"].isin(["Los Angeles", "Lake Placid"])]
usa_not_hosted_medals = usa_not_hosted_medals[usa_not_hosted_medals["Country"] == "USA"]

usa_not_hosted = usa_not_hosted_medals.groupby("Year")["Medal"].count()
usa_not_hosted.sort_values()

Year
1984      9
1994     19
1896     20
1906     24
1998     34
2006     52
1900     63
2014     64
1908     65
2002     84
2010     97
1928    102
1912    107
1936    112
1956    149
1960    152
1952    164
1948    168
1968    173
1976    175
1964    177
1924    194
1920    194
1972    195
1988    214
1992    238
2000    242
2012    248
1996    259
2004    263
2016    264
2008    317
1904    394
Name: Medal, dtype: int64

## **Let's anaylse the medal tally of all the countries together**

## Medal tally of these countries when hosting and non hosting

In [17]:
hosted_medals = pd.DataFrame()
non_hosted_medals = pd.DataFrame()

for countries_cities in countries_cities_map:
  selected_medals = selected_countries_medals[selected_countries_medals["City"].isin(countries_cities_map[countries_cities])]
  hosted_medals = pd.concat([hosted_medals, selected_medals[selected_medals["Country"] == countries_cities]], axis=0).reset_index(drop=True)

  non_selected_medals = selected_countries_medals[~selected_countries_medals["City"].isin(countries_cities_map[countries_cities])]
  non_hosted_medals = pd.concat([non_hosted_medals, non_selected_medals[non_selected_medals["Country"] == countries_cities]], axis=0).reset_index(drop=True)

In [18]:
host_medals = hosted_medals.groupby(["Country", "Year"])["Medal"].count().values

In [19]:
host_medals

array([ 17,   7, 235, 110,  48, 102,  31, 190,   5,  12,  28, 368,  61,
       126, 223,  30, 352])

In [20]:
non_host_medals = non_hosted_medals.groupby(["Country", "Year"])["Medal"].count().values
len(host_medals)

17

## Statistical Testing
Null Hypothesis(H0) : There is no effect of hosting to medal tally.

Alternative Hypothesis(H1) : There is an effect of hosting to medal tally.

In [21]:
from scipy import stats

ttest,pval = stats.ttest_rel(host_medals, non_host_medals[:17])
print(pval)
if pval<0.05:
    print("reject null hypothesis")
else:
    print("accept null hypothesis")

0.0034208312523273396
reject null hypothesis






>
Based on the statistical tests conducted, it appears that hosting the Olympics correlates with an increase in the likelihood of winning medals. Therefore, we can reject the null hypothesis which posited that hosting the Olympics has no significant impact on a country's performance in terms of medal tally. This suggests that the advantages of being a host nation may indeed play a role in enhancing the competitive performance of athletes.



